In [ ]:
!git clone https://github.com/DepthAnything/Depth-Anything-V2
%cd Depth-Anything-V2
!pip install -r requirements.txt
!pip install mediapipe
!pip install git+https://github.com/huggingface/transformers.git
!pip install mediapipe

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-tvaw5htl
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-tvaw5htl
  Resolved https://github.com/huggingface/transformers.git to commit dd16acb8a3e93b643aa374c9fb80749f5235c1a6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
import cv2
import torch
from transformers import DepthProImageProcessorFast, DepthProForDepthEstimation , ZoeDepthForDepthEstimation , DPTForDepthEstimation

# from depth_anything_v2.dpt import DepthAnythingV2

from transformers import AutoImageProcessor, DPTImageProcessor

from PIL import Image
import numpy as np
import mediapipe as mp
from google.colab import files
import gc
gc.collect()
torch.cuda.empty_cache()


mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)


In [5]:
def estimate_depth_depthpro(frame):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    image_processor = DepthProImageProcessorFast.from_pretrained("apple/DepthPro-hf")
    model = DepthProForDepthEstimation.from_pretrained("apple/DepthPro-hf").to(device)

    # Convert frame to PIL Image
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    inputs = image_processor(images=pil_image, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)
    post_processed_output = image_processor.post_process_depth_estimation(
        outputs, target_sizes=[(pil_image.height, pil_image.width)]
    )

    # Extract and normalize depth map
    depth = post_processed_output[0]["predicted_depth"]
    depth = (depth - depth.min()) / (depth.max() - depth.min())
    depth = depth * 255
    depth = depth.detach().cpu().numpy().astype(np.uint8)
    return depth

In [6]:
def estimate_depth_depthanything(frame):

    DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
    model_configs = {
        'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
        'vitb': {'encoder': 'vitb', 'features': 128, 'out_channels': [96, 192, 384, 768]}
    }
    depth_anything = DepthAnythingV2(**model_configs['vits'])
    depth_anything.load_state_dict(torch.load("/content/Depth-Anything-V2/assets/examples_video/depth_anything_v2_vits.pth"), strict=True)
    depth_anything = depth_anything.to(DEVICE).eval()

    # Infer depth from the frame
    depth = depth_anything.infer_image(frame, input_size=518)
    depth = (depth - depth.min()) / (depth.max() - depth.min()) * 255.0
    depth = depth.astype(np.uint8)
    return depth

In [7]:
def estimate_depth_zoedepth(frame):

    image_processor = AutoImageProcessor.from_pretrained("Intel/zoedepth-nyu-kitti")
    model = ZoeDepthForDepthEstimation.from_pretrained("Intel/zoedepth-nyu-kitti")

    # Convert frame to PIL Image
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(frame_rgb)
    inputs = image_processor(images=pil_image, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)

    post_processed_output = image_processor.post_process_depth_estimation(
        outputs, source_sizes=[(pil_image.height, pil_image.width)]
    )

    # Extract and normalize depth map
    predicted_depth = post_processed_output[0]["predicted_depth"]
    depth = (predicted_depth - predicted_depth.min()) / (predicted_depth.max() - predicted_depth.min())
    depth = depth.detach().cpu().numpy() * 255
    depth = depth.astype(np.uint8)
    return depth

In [8]:
def estimate_depth_imads(frame):
    image_processor = DPTImageProcessor.from_pretrained("Intel/dpt-hybrid-midas")
    model = DPTForDepthEstimation.from_pretrained("Intel/dpt-hybrid-midas", low_cpu_mem_usage=True)

    # Convert frame to PIL Image
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(frame_rgb)
    inputs = image_processor(images=pil_image, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)

    prediction = torch.nn.functional.interpolate(
        outputs.predicted_depth.unsqueeze(1),
        size=pil_image.size[::-1],
        mode="bicubic",
        align_corners=False,
    )

    output = prediction.squeeze().cpu().numpy()
    formatted = (output * 255 / np.max(output)).astype("uint8")
    return formatted

In [9]:
def estimate_pose_mideapipe(frame):
    import cv2
    import mediapipe as mp

    # Initialize MediaPipe Pose
    mp_pose = mp.solutions.pose
    with mp_pose.Pose(static_image_mode=False, model_complexity=2, min_detection_confidence=0.5) as pose:
        results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        keypoints = {}

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # Extract foot keypoints
            keypoints["left_foot"] = (
                int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].x * frame.shape[1]),
                int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].y * frame.shape[0])
            )
            keypoints["right_foot"] = (
                int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].x * frame.shape[1]),
                int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].y * frame.shape[0])
            )

            # Extract shoulder keypoints
            keypoints["left_shoulder"] = (
                int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x * frame.shape[1]),
                int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y * frame.shape[0])
            )
            keypoints["right_shoulder"] = (
                int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * frame.shape[1]),
                int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * frame.shape[0])
            )

        return keypoints

In [10]:
# 1. Calculate 2D Euclidean Distance (No Depth)
def calculate_2D_euclidean_distance_no_focal(left_foot_2d, right_foot_2d):
    """
    Calculate the Euclidean distance between two points in 2D space without using depth.
    :param left_foot_2d: 2D coordinates of the left foot (x, y).
    :param right_foot_2d: 2D coordinates of the right foot (x, y).
    :return: Euclidean distance between the two points.
    """
    if left_foot_2d is not None and right_foot_2d is not None:
        foot_distance_2d = np.linalg.norm(np.array(left_foot_2d) - np.array(right_foot_2d))
        return foot_distance_2d
    return None

In [11]:
# def get_3d_point(landmark, depth_map):
#     x, y = landmark
#     if 0 <= y < depth_map.shape[0] and 0 <= x < depth_map.shape[1]:
#         z = depth_map[y, x]
#     else:
#         return None
#     return np.array([x, y, z])

In [12]:
# 2. Calculate 3D Euclidean Distance (With Depth, No Focal Length)
def calculate_3D_euclidean_distance_no_focal(left_foot_2d, right_foot_2d, depth_map):
    """
    Calculate the Euclidean distance between two points in 3D space without using focal length.
    :param left_foot_2d: 2D coordinates of the left foot (x, y).
    :param right_foot_2d: 2D coordinates of the right foot (x, y).
    :param depth_map: Depth map corresponding to the frame.
    :return: Euclidean distance between the two points.
    """
    def get_3d_point(landmark, depth_map):
        x, y = landmark
        if 0 <= y < depth_map.shape[0] and 0 <= x < depth_map.shape[1]:
            z = depth_map[y, x]
        else:
            return None
        return np.array([x, y, z])

    left_foot_3d = get_3d_point(left_foot_2d, depth_map)
    right_foot_3d = get_3d_point(right_foot_2d, depth_map)

    if left_foot_3d is not None and right_foot_3d is not None:
        foot_distance_3d = np.linalg.norm(left_foot_3d - right_foot_3d)
        return foot_distance_3d
    return None

In [13]:
# 3. Estimate Focal Length
def estimate_focal_length(shoulder_width_pixels, shoulder_depth, shoulder_width_cm=60):
    """
    Estimate the focal length using the shoulder width and depth.
    :param shoulder_width_pixels: Width of the shoulders in pixels.
    :param shoulder_depth: Depth (distance from the camera) of the shoulders in cm.
    :param shoulder_width_cm: Real-world shoulder width in cm (default: 60 cm).
    :return: Estimated focal length in pixels.
    """
    if shoulder_depth <= 0:
        print("Warning: Invalid shoulder depth detected.")
        return None
    focal_length = (shoulder_width_pixels * shoulder_depth) / shoulder_width_cm
    return focal_length

In [14]:
# 4. Calculate Real-World 3D Distance (With Depth and Focal Length)
def calculate_real_world_distance(left_foot_2d, right_foot_2d, depth_map, focal_length, image_shape):
    """
    Calculate the real-world 3D distance between two points using depth and focal length.
    :param left_foot_2d: 2D coordinates of the left foot (x, y).
    :param right_foot_2d: 2D coordinates of the right foot (x, y).
    :param depth_map: Depth map corresponding to the frame.
    :param focal_length: Focal length of the camera (in pixels).
    :param image_shape: Shape of the image (height, width, channels).
    :return: Real-world distance between the two points in centimeters.
    """
    height, width = image_shape[:2]

    def get_3d_point_with_focal(landmark, depth_map, focal_length, image_shape):
        x, y = int(landmark[0] * width), int(landmark[1] * height)
        if 0 <= y < height and 0 <= x < width:
            depth = depth_map[y, x]
        else:
            return None
        z = depth
        if focal_length is not None and focal_length > 0:
            x_3d = (x - width / 2) * z / focal_length
            y_3d = (y - height / 2) * z / focal_length
        else:
            x_3d, y_3d = 0, 0
        return np.array([x_3d, y_3d, z])

    left_foot_3d = get_3d_point_with_focal(left_foot_2d, depth_map, focal_length, image_shape)
    right_foot_3d = get_3d_point_with_focal(right_foot_2d, depth_map, focal_length, image_shape)

    if left_foot_3d is not None and right_foot_3d is not None:
        foot_distance_3d = np.linalg.norm(left_foot_3d - right_foot_3d)
        return foot_distance_3d * 100  # Convert meters to centimeters
    return None

In [15]:
def test_framework(video_path, depth_model_name, pose_model="mideapipe", focal_length=None):
    import cv2
    import numpy as np

    # Load depth estimation model
    depth_models = {
        "depthpro": estimate_depth_depthpro,
        "depthanything": estimate_depth_depthanything,
        "zoedepth": estimate_depth_zoedepth,
        "imads": estimate_depth_imads
    }
    depth_model = depth_models.get(depth_model_name)
    if not depth_model:
        raise ValueError(f"Unknown depth model: {depth_model_name}")

    # Load pose estimation model
    def estimate_pose(frame):
        return estimate_pose_mideapipe(frame)

    # Open video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video file.")
        return

    # Output video writer setup
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_path = 'output_with_distances.mp4'
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Estimate depth
        depth_map = depth_model(frame)

        # Estimate pose
        keypoints = estimate_pose(frame)

        # Extract 2D coordinates of feet
        if "left_foot" in keypoints and "right_foot" in keypoints:
            left_foot_2d = keypoints["left_foot"]
            right_foot_2d = keypoints["right_foot"]

            # Calculate distances using all three methods
            distance_2d = calculate_2D_euclidean_distance_no_focal(left_foot_2d, right_foot_2d)
            distance_3d_no_focal = calculate_3D_euclidean_distance_no_focal(left_foot_2d, right_foot_2d, depth_map)

            # Estimate focal length if the model is not DepthPro and focal length is not provided
            if depth_model_name != "depthpro" and focal_length is None:
              print("Estimating focal length")
              if "left_shoulder" in keypoints and "right_shoulder" in keypoints:
                left_shoulder = keypoints["left_shoulder"]
                right_shoulder = keypoints["right_shoulder"]

                # Calculate shoulder width in pixels
                shoulder_width_pixels = np.linalg.norm(np.array(left_shoulder) - np.array(right_shoulder))
                # Extract shoulder depth (average depth of shoulder points)
                shoulder_depth_left = depth_map[left_shoulder[1], left_shoulder[0]] if 0 <= left_shoulder[1] < depth_map.shape[0] and 0 <= left_shoulder[0] < depth_map.shape[1] else None
                shoulder_depth_right = depth_map[right_shoulder[1], right_shoulder[0]] if 0 <= right_shoulder[1] < depth_map.shape[0] and 0 <= right_shoulder[0] < depth_map.shape[1] else None

                if shoulder_depth_left is not None and shoulder_depth_right is not None:
                        shoulder_depth = (shoulder_depth_left + shoulder_depth_right) / 2
                        focal_length = estimate_focal_length(shoulder_width_pixels, shoulder_depth)
                        print(f"Estimated Focal Length: {focal_length:.2f} pixels")
                    else:
                        print("Warning: Could not estimate focal length due to missing shoulder depth.")

            # Calculate real-world distance if focal length is available
            if focal_length is not None:
                distance_3d_real_world = calculate_real_world_distance(
                    left_foot_2d, right_foot_2d, depth_map, focal_length, frame.shape
                )
            else:
                distance_3d_real_world = None

            # Draw lines and display distances
            if distance_2d is not None:
                cv2.line(frame, tuple(left_foot_2d), tuple(right_foot_2d), (0, 255, 0), 2)
                text_position = (10, 30)
                cv2.putText(frame, f"2D Distance: {distance_2d:.2f}", text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            if distance_3d_no_focal is not None:
                text_position = (10, 70)
                cv2.putText(frame, f"3D Distance (No Focal): {distance_3d_no_focal:.2f}", text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            if distance_3d_real_world is not None:
                text_position = (10, 110)
                cv2.putText(frame, f"Real-World Distance: {distance_3d_real_world:.2f} cm", text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        else:
            print("Pose keypoints not detected.")

        # Write the processed frame to the output video
        out.write(frame)

        # Print progress
        frame_count += 1
        if frame_count % 10 == 0:  # Print every 10 frames
            print(f"Processed {frame_count} frames.")

    # Release resources
    cap.release()
    out.release()
    files.download(output_path)
    print("Processing complete. Output video saved as:", output_path)

In [17]:
# Load depth estimation model
# depth_models = {
#     "depthpro": estimate_depth_depthpro,
#     "depthanything": estimate_depth_depthanything,
#     "zoedepth": estimate_depth_zoedepth,
#     "imads": estimate_depth_imads
# }
video_path = "/content/Kirolos_video.mp4"
depth_model_name = "imads"

test_framework(video_path, depth_model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/382 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/9.88k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/490M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/490M [00:00<?, ?B/s]

<ipython-input-15-51413abf0dad>:70: RuntimeWarning: overflow encountered in scalar add
  shoulder_depth = (shoulder_depth_left + shoulder_depth_right) / 2


Estimated Focal Length: 360.53 pixels
Processed 10 frames.
Processed 20 frames.
Processed 30 frames.
Processed 40 frames.
Processed 50 frames.
Processed 60 frames.
Processed 70 frames.
Processed 80 frames.
Processed 90 frames.
Processed 100 frames.
Processed 110 frames.
Processed 120 frames.
Processed 130 frames.
Processed 140 frames.
Processed 150 frames.
Processed 160 frames.
Processed 170 frames.
Processed 180 frames.
Processed 190 frames.
Processed 200 frames.
Processed 210 frames.
Processed 220 frames.
Processed 230 frames.
Processed 240 frames.
Processed 250 frames.
Processed 260 frames.
Processed 270 frames.
Processed 280 frames.
Processed 290 frames.
Processed 300 frames.
Processed 310 frames.
Processed 320 frames.
Processed 330 frames.
Processed 340 frames.
Processed 350 frames.
Processed 360 frames.
Processed 370 frames.
Processed 380 frames.
Processed 390 frames.
Processed 400 frames.
Processed 410 frames.
Processed 420 frames.
Processed 430 frames.
Processed 440 frames.
Pro

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Processing complete. Output video saved as: output_with_distances.mp4
